In [4]:
# LIBRARY USED TO BUILD
import gradio as gr
import logging
from binance.client import Client
from binance.exceptions import BinanceAPIException

#  LOGGING CONFIG
logging.basicConfig(
    filename="trading_bot.log",
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO
)

# BOT CLASS 
class BasicBot:
    def __init__(self, api_key, api_secret, testnet=True):
        self.client = Client(api_key.strip(), api_secret.strip())
        if testnet:
            self.client.FUTURES_URL = "https://testnet.binancefuture.com"

    # Check balance
    def get_balance(self, asset='USDT'):
        try:
            balances = self.client.futures_account_balance()
            for b in balances:
                if b['asset'] == asset:
                    return b
            return {'asset': asset, 'balance': 0}
        except Exception as e:
            logging.error(f"Balance check failed: {e}")
            return {'error': str(e)}

    # Place orders
    def place_order(self, symbol, side, order_type, quantity, price=None, stop_price=None):
        try:
            if order_type.upper() == "MARKET":
                order = self.client.futures_create_order(
                    symbol=symbol.upper(),
                    side=side.upper(),
                    type="MARKET",
                    quantity=quantity
                )

            elif order_type.upper() == "LIMIT":
                if price is None:
                    return {"error": "Limit price required for LIMIT order"}
                order = self.client.futures_create_order(
                    symbol=symbol.upper(),
                    side=side.upper(),
                    type="LIMIT",
                    timeInForce="GTC",
                    quantity=quantity,
                    price=str(price)
                )

            elif order_type.upper() == "STOP_MARKET":
                if stop_price is None:
                    return {"error": "Stop price required for STOP_MARKET order"}
                order = self.client.futures_create_order(
                    symbol=symbol.upper(),
                    side=side.upper(),
                    type="STOP_MARKET",
                    stopPrice=str(stop_price),
                    quantity=quantity
                )
            else:
                return {"error": f"Order type {order_type} not supported"}

            logging.info(f"Order placed: {order}")
            return order

        except BinanceAPIException as e:
            logging.error(f"Binance API Exception: {e}")
            return {"error": f"Binance API Exception: {e}"}
        except Exception as e:
            logging.error(f"Order failed: {e}")
            return {"error": str(e)}

#  GRADIO FUNCTIONS
bot_instance = None

def connect(api_key, api_secret):
    global bot_instance
    try:
        bot_instance = BasicBot(api_key, api_secret, testnet=True)
        balance = bot_instance.get_balance('USDT')
        return f"Connected to Binance Testnet!\nUSDT Balance: {balance.get('balance', 'N/A')}"
    except Exception as e:
        return f"Connection failed: {str(e)}"

def place_trade(symbol, side, order_type, quantity, price, stop_price):
    global bot_instance
    if bot_instance is None:
        return "Bot not connected. Enter API credentials first."
    
    try:
        # Convert quantity, price, stop_price to float if not empty
        qty = float(quantity)
        prc = float(price) if price else None
        sp = float(stop_price) if stop_price else None

        result = bot_instance.place_order(symbol, side, order_type, qty, prc, sp)
        return str(result)
    except Exception as e:
        return f"Trade failed: {str(e)}"

#  GRADIO INTERFACE 
with gr.Blocks() as demo:
    gr.Markdown("## Binance Futures Testnet Trading Bot")

    with gr.Tab("Connect"):
        api_key_input = gr.Textbox(label="API Key", type="password")
        api_secret_input = gr.Textbox(label="API Secret", type="password")
        connect_btn = gr.Button("Connect to Testnet")
        connect_output = gr.Textbox(label="Connection Status")
        connect_btn.click(connect, inputs=[api_key_input, api_secret_input], outputs=connect_output)

    with gr.Tab("Trade"):
        symbol_input = gr.Textbox(label="Trading Pair (e.g., BTCUSDT)")
        side_input = gr.Dropdown(label="Side", choices=["BUY", "SELL"])
        order_type_input = gr.Dropdown(label="Order Type", choices=["MARKET", "LIMIT", "STOP_MARKET"])
        quantity_input = gr.Textbox(label="Quantity")
        price_input = gr.Textbox(label="Price (for LIMIT orders)")
        stop_price_input = gr.Textbox(label="Stop Price (for STOP_MARKET orders)")
        trade_btn = gr.Button("Place Order")
        trade_output = gr.Textbox(label="Order Response")
        trade_btn.click(
            place_trade, 
            inputs=[symbol_input, side_input, order_type_input, quantity_input, price_input, stop_price_input],
            outputs=trade_output
        )

demo.launch()


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


BONUS
1.TO PLACE THE MARKET ORDER 
Trading Pair: BTCUSDT

Side: BUY or SELL

Order Type: MARKET

Quantity: e.g., 0.001 BTC

Price: leave empty

Stop Price: leave empty

2.TO PLACE A LIMIT ORDER
Trading Pair: BTCUSDT

Side: BUY or SELL

Order Type: LIMIT

Quantity: e.g., 0.001 BTC

Price: e.g., 30000 (whatever price you want to buy/sell at)

Stop Price: leave empty

Click Place Order



3.PLACE a STOP_MARKET ORDER
Trading Pair: BTCUSDT

Side: BUY or SELL

Order Type: STOP_MARKET

Quantity: e.g., 0.001 BTC

Price: leave empty

Stop Price: e.g., 31000

Click Place Order